# This is a scratch space for building functions
After functions are made, move them to the corresponding folder. 

First we import the necessary objects:

In [ ]:
from pathlib import Path
from flow_analysis_comps.PIV.PIV_process import AMF_PIV
from flow_analysis_comps.PIV.definitions import PIV_params
from flow_analysis_comps.PIV.PIV_visualize import visualizerParams
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget
%load_ext autoreload
%autoreload 2

## Video import and general settings
Videos are assumed to have the following structure:

```
videoFolder
|--videoInfo.txt / metadata.json
|--Img
    |--Img_****.tif
```

Such that the metadata is not in the same folder as the TIF files. 

In [ ]:
data_adr = Path(
    r"C:\Users\vstaalduine\AMOLF-SHIMIZU Dropbox\Simon van Staalduine\035"
)

winsize = 64  # pixels, interrogation window size in frame A
nr_of_passes = 3
STN_thresh = 0.001
video_path = str(data_adr / "Img")


piv_param_obj = PIV_params(
    root_path=data_adr,
    segment_mode="brightfield",
    window_size_start=winsize,
    number_of_passes=nr_of_passes,
    stn_threshold=STN_thresh,
    max_speed_px_per_frame=100.0,
)

PIV_process = AMF_PIV(piv_param_obj)


In [ ]:
PIV_process.plot_raw_images((0, 1))

In [ ]:
PIV_process.run_single_frame((15, 16))

In [ ]:
PIV_process.run_full_video()

In [ ]:
PIV_process.start_visualizer(
    data_adr / "OpenPIV_results_16_PIV_output", limit_data=False
)

In [ ]:
mean_orientation = PIV_process.visualizer.get_mean_generic(
    "speed_dir", IS_MEAN_CIRCULAR=True
)
mean_speed = PIV_process.visualizer.get_mean_generic("abs")

mean_orientation = (mean_orientation + np.pi) % (2 * np.pi)

In [ ]:
import colorcet as cc
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

norm = plt.Normalize(vmin=0, vmax=2*np.pi)
cmap = cc.cm.CET_C7

rgb_image = cmap(norm(mean_orientation))[:,:,:3]
hsv_image = rgb_to_hsv(rgb_image)
hsv_image[:,:,2]  *= (mean_speed / mean_speed.max())
# rgb_image = hsv_to_rgb(hsv_image)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(
    rgb_image,
    origin="lower",
    extent=[
        0,
        PIV_process.visualizer.current_frame_data["x"].max(),
        0,
        PIV_process.visualizer.current_frame_data["y"].max(),
    ],
)
ax[0].set_title("Orientation")
ax[1].imshow(
    mean_speed,
    origin="lower",
    vmin=0,
    cmap="cet_CET_L16",
    extent=[
        0,
        PIV_process.visualizer.current_frame_data["x"].max(),
        0,
        PIV_process.visualizer.current_frame_data["y"].max(),
    ],
)
ax[1].set_title("Speed magnitude")

In [ ]:
PIV_process.visualizer.set_image_index(377
                                       )
fig = PIV_process.visualizer.plot_full_figure(dpi=70)

In [ ]:
# PIV_process.visualizer.set_image_index(450)
dist_thresh = 30

result, result2 = PIV_process.visualizer.speed_against_point_distance((64, 68))
dist_r = PIV_process.visualizer.interpolate_from_dataframe(result)

fig1, ax1 = plt.subplots()
ax1.imshow(dist_r < dist_thresh, origin="lower",     extent=[
        0,
        PIV_process.visualizer.current_frame_data["x"].max(),
        0,
        PIV_process.visualizer.current_frame_data["y"].max(),
    ],)

fig, ax = plt.subplots()
ax.scatter(result, result2, c=PIV_process.visualizer.current_frame_data["speed_dir"], cmap=cc.cm.CET_C7)
ax.set_xlim(0, dist_thresh)


In [ ]:
PIV_process.visualizer.show_mean_quiver_plot(scale=10)